In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from scipy.stats import skew, kurtosis

The datashare file contains all the time-series stocks data of all the U.S. markets, including NYSE, NASDAQ and AMEX. 
every stock also contains all the corresponding stock characteristics that are used as predictors in the models.

## NOTE

Given the size of the origial dataset is too big to be imported on Github, the importing on the data has to be done locally. The Notebook provided includes all the steps we did in the datapreprocessing, but in order for the notebook to run smoothly it is necessary to dowload the original dataset first (a link is provided).

[Stocks Data Dowlnoad](https://dachxiu.chicagobooth.edu/)

In [2]:
Stocks = pd.read_csv("datashare.csv")

In [3]:
Stocks.index = Stocks["DATE"]
Stocks.index = pd.to_datetime(Stocks.index, format="%Y%m%d")

In [4]:
Stocks = Stocks.drop("DATE", axis=1)

In [ ]:
Stocks = Stocks['1957-03':"2016"] 
Stocks['Month'] = Stocks.index
Stocks

## 1.1 Stocks Selection

In [6]:
top500_df = Stocks.groupby(Stocks.index).apply(lambda x: x.nlargest(500, 'mvel1')).reset_index(drop=True)

In [7]:
top500_df.index = top500_df["Month"]

In [ ]:
top500_df

In [9]:
top500_df= top500_df.rename_axis('Date')

In [ ]:
top500_df

### List with Stock Characteristics

In [11]:
characteristics_l = list(set(top500_df.columns).difference({'permno', 'Month', 'sic2', 'weight', 'total_market_cap'}))
len(characteristics_l)

94

## 1.2 Adding Stock Returns

In this section we add to the dataframe the returns of every stock in the corresponding point in time. The data is retrieved from the CRSP databse (via WRDS) based on the PERMNO number of the stock. 

In [ ]:
Rets = pd.read_csv("Permno_date_return.csv")
Rets['date'] = pd.to_datetime(Rets['date'])
Rets.index = Rets['date']
Rets.rename({'date':'Month'}, inplace=True, axis=1)
Rets

In [ ]:
Rets.rename({'PERMNO':'permno'}, inplace=True, axis=1)
Rets

In [ ]:
merged = pd.merge(top500_df, Rets, on=['permno', 'Month'])
merged.index = merged['Month']
merged = merged.rename_axis('Date')
merged

## 1.3 Removing stocks without returns data

In [ ]:
merged = merged.dropna(subset=['RET'])
merged

After removing observations without stock returns a total of 643 rows is removed (0.18%).

## 1.4 Stock-Weighting

In this section we use the stock market cap to compute the corresponding weight in that point in time for every company in the replicating portfolio (S&P500). 

In [ ]:
merged['Month'] = pd.to_datetime(merged['Month'])

In [17]:
total_market_cap = merged.groupby(merged['Month'].dt.to_period("M"))['mvel1'].sum()
merged = merged.merge(total_market_cap.rename('total_market_cap'), left_on=merged['Month'].dt.to_period("M"), right_index=True)

In [ ]:
merged['weight'] = merged['mvel1'] / merged['total_market_cap']
merged = merged.drop('key_0', axis=1)
merged

In [19]:
merged['Month'] = merged['Month'].dt.strftime('%Y-%m')
merged.index = merged['Month']

In [ ]:
merged = merged.rename_axis('Date')
merged

## 2.1 Creating Dummy variables for SIC2 characteristic

In this section we compiute the dummy variables for the industry code (variable) "SIC2"

In [ ]:
dummy_variables = pd.get_dummies(merged['sic2'], prefix='SIC')
merged_dum = pd.concat([merged, dummy_variables], axis=1)

merged_dum

In [ ]:
merged_dum['sic2']

In the paper 74 dummies are obtained, in our case only 65 since we are reducing the analysis only to the firms contained in the S&P500 in the time period considered, which results in 9 less industry dummies.

## 2. Replicating Portfolio - Returns

In this section we compute the (weighted) monthly return of every stock in order to compute the return of the replicating portfolio in every period. This step will also allow to check the correlation or the replicating portfolio returns with the ones of the index, to check the accuracy of the portfolio with the benchmark. 

In [ ]:
merged['RET'] = merged['RET'].astype(float)
merged['weighted_RET'] = merged['weight'] * merged['RET']
replicating_returns = merged.groupby('Date')['weighted_RET'].sum().reset_index()
replicating_returns.index = replicating_returns['Date']
replicating_returns = replicating_returns.drop('Date', axis=1)
replicating_returns

## 3.1 Macro Predictors

The data source is professor's Amid Goyal's personal website, but this dataset has been included in the Github folder under the name "PredictorData2022".

According to the paper, firm characteristics are lagged due to the data being released with a delay. To match this we lag macro predictors by one month.

In [ ]:
Macro_pred = pd.read_csv('PredictorData2022.csv', parse_dates=True, index_col=0)
Macro_pred.index = pd.to_datetime(Macro_pred.index, format="%Y%m").to_period('M')
Macro_pred_lag = Macro_pred.shift(1) #lag values by one month
Macro_pred_lag = Macro_pred_lag.rename_axis('Date')
Macro_pred_lag = Macro_pred_lag['1957-03':'2016']
Macro_pred_lag

In [ ]:
Macro_pred= Macro_pred.rename_axis('Date')
Macro_pred = Macro_pred['1957-03':'2016']
Macro_pred

#### Macroeconomic Factors: 
* including dividend-price ratio (dp): (d/p) is the difference between the log of dividends and the log of prices.
* earnings-price ratio (ep): (e/p) is the difference between the log of earnings and the log of prices.
* book-to-market ratio (bm)
* net equity expansion (ntis)
* Treasury-bill rate (tbl)
* term spread (tms): The Term Spread (tms) is the difference between the long term yield on government bonds (lty) and the Treasury-bill (tbl)
* default spread (dfy): The Default Yield Spread (dfy) is the difference between BAA and AAA-rated corporate bond yields.
* stock variance (svar)

In [26]:
Macro_pred_lag['Index'] = Macro_pred_lag['Index'].str.replace(',', '').astype(float)
Macro_pred_lag["d/p"] = np.log(Macro_pred_lag["D12"]) - np.log(Macro_pred_lag["Index"])
Macro_pred_lag["e/p"] = np.log(Macro_pred_lag["E12"]) - np.log(Macro_pred_lag["Index"])
Macro_pred_lag["tms"] = Macro_pred_lag["lty"] - Macro_pred_lag["tbl"]
Macro_pred_lag['dfy'] = Macro_pred_lag['BAA'] - Macro_pred_lag['AAA']

Only the macroeconomic varibales used in the paper are selected:

In [ ]:
# drop the redundent variables - Selecting the needed Macro predictors used in the paper
Macro_pred_sel = Macro_pred_lag.drop(columns=['Index','D12', 'E12', 'AAA', 'BAA', 'CRSP_SPvwx', 'corpr','Rfree', 'CRSP_SPvw', 'lty','infl', 'ltr','csp'])
Macro_pred_sel

### Creating list with Macro Predictors

In [28]:
Macro_pred_l = ['b/m', 'tbl', 'ntis', 'svar', 'd/p', 'e/p', 'tms', 'dfy']

In [ ]:
# Create the 'months' column from the existing DatetimeIndex
Macro_pred_sel['Month'] = Macro_pred_sel.index.strftime('%Y-%m')

# Display the DataFrame to verify the new column
print(Macro_pred_sel.head())


## 4.1 Reliability of Replicating Portfolio - S&P500 Sanity Check

**Correlation:**

In [30]:
np.corrcoef(replicating_returns['weighted_RET'], Macro_pred['CRSP_SPvw'])

array([[1.        , 0.99543735],
       [0.99543735, 1.        ]])

**Mean:**

In [31]:
abs(np.mean(replicating_returns['weighted_RET'])-np.mean(Macro_pred['CRSP_SPvw']))

0.00027194334994176846

**Variance:**

In [32]:
abs(np.var(replicating_returns['weighted_RET'])-np.var(Macro_pred['CRSP_SPvw']))

9.990573194619459e-06

**Skewness:**

In [33]:
abs(skew(replicating_returns['weighted_RET'])-skew(Macro_pred['CRSP_SPvw']))

0.0639163545294062

**Kurtosis:**

In [34]:
abs(kurtosis(replicating_returns['weighted_RET'])-kurtosis(Macro_pred['CRSP_SPvw']))

0.057543532982029966

## 5.1 Creating Interaction Terms

In [ ]:
merged_macro_char = pd.merge(Macro_pred_sel, merged_dum, on=['Month'])
merged_macro_char.index = merged_macro_char['Month']
merged_macro_char = merged_macro_char.rename_axis('Date')
merged_macro_char

In [ ]:
warnings.filterwarnings("ignore", message="DataFrame is highly fragmented")

In [ ]:
data = merged_macro_char.copy()
for fc in characteristics_l:
    for mp in Macro_pred_l:
        data[fc + '*' + mp] = merged_macro_char[fc] * merged_macro_char[mp]
        
data

## 5.2 Computing Excess Returns

In [37]:
data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
data['tbl'] = pd.to_numeric(data['tbl'], errors='coerce')

# Perform subtraction operation
data['RET'] = data['RET'] - data['tbl']

In [38]:
data.rename(columns={'RET': 'Exc_RET'}, inplace=True)

In [39]:
data=data.drop(columns=Macro_pred_l)

In [ ]:
data

## 5.3 Scaling Variables to [-1,1] and Dataset Creation

In [41]:
from sklearn.preprocessing import MinMaxScaler

features = list(set(data.columns).difference({'permno','Month','Exc_RET','total_market_cap','weight','sic2'})) # a list storing all features

X = MinMaxScaler((-1,1)).fit_transform(data[features])
X = pd.DataFrame(X, columns=features)

In [ ]:
y = data['Exc_RET']
y

In [ ]:
X.index = data.index
X.fillna(0, inplace=True)
X

## 6.1 Exporting Processed Data

If you want to export the processed datasets, delete the '#' symbols from the cell below. Note that it takes a bit of time depending on the computer, generally between 3 and 10 minutes.

In [46]:
X.to_csv('Features_lagged_X.csv')
y.to_csv('Dependent_y.csv')

In [ ]:
weights = pd.DataFrame(merged['weight'])
weights.to_csv('Stocks_weights.csv')